In [1]:
import gymnasium as gym
import panda_gym
from sb3_contrib import TQC
from huggingface_sb3 import load_from_hub
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
import time
import numpy as np
from panda_gym.utils import distance
import random

In [2]:
chk = load_from_hub(repo_id="BanUrsus/tqc-PandaPickAndPlace-v3", filename="tqc-PandaPickAndPlace-v3.zip")
stats = load_from_hub(repo_id="BanUrsus/tqc-PandaPickAndPlace-v3", filename="vec_normalize.pkl")
   
env = gym.make('PandaPickAndPlace-v3', render_mode="human", autoreset=False)

sim = env.sim
robot = env.robot
task = env.task

env = DummyVecEnv([lambda: env])
env = VecNormalize.load(stats, env)
env.training = False
env.norm_reward = False

pybullet build time: Apr 10 2024 13:52:30


argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
Version = 4.1 Metal - 88
Vendor = Apple
Renderer = Apple M3 Pro
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


/Users/alexbriones/Library/Caches/pypoetry/virtualenvs/icaml-planning-fFRb6YwO-py3.11/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.sim to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.sim` for environment variables or `env.get_wrapper_attr('sim')` that will search the reminding wrappers.
  logger.warn(
/Users/alexbriones/Library/Caches/pypoetry/virtualenvs/icaml-planning-fFRb6YwO-py3.11/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.robot to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.robot` for environment variables or `env.get_wrapper_attr('robot')` that will search the reminding wrappers.
  logger.warn(
/Users/alexbriones/Library/Caches/pypoetry/virtualenvs/icaml-planning-fFRb6YwO-py3.11/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.task to g

In [3]:
task.goal_range_high

array([0.15, 0.15, 0.2 ])

In [4]:
model = TQC.load(chk,env) 
env = model.env
obs = env.reset()

q1 = np.array([0.15, 0.15 , 0.0])
q2 = np.array([0.15, -0.15, 0.0])
q3 = np.array([-0.15, 0.15, 0.0])
q4 = np.array([-0.15, -0.15, 0.0])

/Users/alexbriones/Library/Caches/pypoetry/virtualenvs/icaml-planning-fFRb6YwO-py3.11/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


In [5]:
sim.create_box(
    body_name="q1",
    half_extents=np.ones(3) * 0.04 / 2,
    mass=1.0,
    position=q1,
    rgba_color=np.array([1.0, 0.0, 0.0, 1.0]),
)

In [6]:
sim.create_box(
    body_name="q2",
    half_extents=np.ones(3) * 0.04 / 2,
    mass=1.0,
    position=q2,
    rgba_color=np.array([0.5, 0.5, 0.0, 1.0]),
)

In [7]:
sim.create_box(
    body_name="q3",
    half_extents=np.ones(3) * 0.04 / 2,
    mass=1.0,
    position=q3,
    rgba_color=np.array([0.0, 0.0, 1.0, 1.0]),
)

In [8]:
sim.create_box(
    body_name="q4",
    half_extents=np.ones(3) * 0.04 / 2,
    mass=1.0,
    position=q4,
    rgba_color=np.array([0.0, 0.5, 0.5, 1.0]),
)

In [9]:


random_states = []
solved_states = []


# predicates method / helper func
def calculate_proximity(ee_position, block_position):
    # Euclidean distance
    return np.linalg.norm(np.array(ee_position) - np.array(block_position))

def is_gripper_open(finger_width):
    return finger_width > 0.02

def is_gripper_correctly_oriented(ee_position, block_position, z_threshold=0.025):
    return abs(ee_position[2] - block_position[2]) <= z_threshold 

def is_block_grasped(finger_width, proximity):
    return finger_width <= 0.04 and finger_width >= 0.02 and proximity < 0.05  



# TODO
# Change Defn of is block is being gripped 
    

def is_sliding(block_position, goal_position, z_threshold=0.02):
    # Check if the block and goal are on the same z-plane within a threshold
    return abs(block_position[2] - goal_position[2]) <= z_threshold

while len(random_states) < 10000 or len(solved_states) < 1000:
    action, _states = model.predict(obs, deterministic=True)
    # print(action)
    obs, reward, dones, info = env.step(action)

    # gather data
    ee = robot.get_ee_position()
    fw = robot.get_fingers_width()
    block = task.get_achieved_goal()
    goal = np.array(sim.get_base_position("target"))

    # predicates
    distance_to_block = calculate_proximity(ee, block)
    gripper_open = is_gripper_open(fw)
    gripper_correctly_oriented = is_gripper_correctly_oriented(ee,block)
    block_grasped = is_block_grasped(fw, distance_to_block)
    sliding = is_sliding(block,goal)

    # abstract state
    state = {
        "effector_pos": ee,
        "finger_width": fw,
        "gripper_open": gripper_open,
        "gripper_orientation_correct": gripper_correctly_oriented,
        "block_grasped": block_grasped,
        "d_q1": distance(ee, q1),
        "d_q2": distance(ee, q2),
        "d_q3": distance(ee, q3),
        "d_q4": distance(ee, q4),
        "distance_to_block": distance_to_block,
        "block_to_goal": distance(block, goal),
        "is_grasping": fw > 0.02 and fw < 0.04,
        "is_closed": fw < 0.01,
        "sliding": sliding,
        "finished": dones[0]
    }

    # check matching w/ screen
    # current state output of abstract states
    print("Current State:")
    for key, value in state.items():
        print(f"{key}: {value}")

    time.sleep(10.5)

    if dones[0]:
        solved_states.append(state)
        obs = env.reset()
    else:
        if random.randint(0, 10) == 0:
            random_states.append(state)

print(f"finished: {len(random_states)} random states, {len(solved_states)} solved states")

Current State:
effector_pos: [0.02259773 0.02277998 0.1655107 ]
finger_width: 0.07027377658727625
gripper_open: True
gripper_orientation_correct: False
block_grasped: False
d_q1: 0.24456096500818172
d_q2: 0.2710683542170242
d_q3: 0.27086657511489226
d_q4: 0.29501981046476283
distance_to_block: 0.16284085472258736
block_to_goal: 0.14636724686055966
is_grasping: False
is_closed: False
sliding: False
finished: False
Current State:
effector_pos: [-0.00990335  0.04207787  0.11278058]
finger_width: 0.07881771565827705
gripper_open: True
gripper_orientation_correct: False
block_grasped: False
d_q1: 0.2234630291820425
d_q2: 0.2741941768240946
d_q3: 0.20974679436443902
d_q4: 0.2631357804236944
distance_to_block: 0.1008138231908829
block_to_goal: 0.14636762841652534
is_grasping: False
is_closed: False
sliding: False
finished: False
Current State:
effector_pos: [-0.0251868   0.03147452  0.06264944]
finger_width: 0.07985601221696101
gripper_open: True
gripper_orientation_correct: False
block_grasp

KeyboardInterrupt: 

In [ ]:
# robot.get_ee_position() gets us the position of the robots "fingers"
# robot.get_fingers_width() gets us the width between the robots "fingers"

# the table has dimensions (1.1, 0.7, 0.4) w/ x_offset=-0.3
# can partition the table into 4 quadrants
# q1 = (1.1 / 4 - 0.3, 0.7 / 4 , 0.4)
# q2 = (1.1 / 4 - 0.3, 3 * (0.7 / 4), 0.4)
# q3 = ((1.1 / 4) * 3 - 0.3, 0.7 / 4, 0.4)
# q4 = ((1.1 / 4) * 3 - 0.3, 3 * (0.7 / 4), 0.4)

# distance_to_q{} = d(robot.get_ee_position(), q{})

# grasping might be distance from effectors to block & a check to see if they arent completely open or closed
# distance_to_block = d(robot.get_ee_position(), desired_goal)


In [ ]:
# task.get_achieved_goal() where the block is now
# desired_goal = np.array(sim.get_base_position("target"))
# d = distance(achieved_goal, desired_goal)

In [ ]:
# looks like overall we keep the env in the wrapped form and just save the variables
# so we can mess with them elsewhere